In [2]:
pip install biogeme

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
import biogeme.version as ver

In [4]:
print(ver.getText())

biogeme 3.2.12 [2023-11-23]
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [5]:
import pandas as pd
import biogeme.biogeme as bio
import biogeme.database as db
from biogeme import models
from biogeme.expressions import Beta
import numpy as np

In [7]:
# load the data
# if loading CSV file 
# df_Car_SP_data = pd.read_csv('Car_SP_data.csv')

df=pd.read_csv('data.dat',delimiter='\t')

In [8]:
df  # making sure missing values can be recorded

,Individual,Children_u16,Car_travel_time,PT_travel_time,Chosen_alternative
0,1,1,28,48,0
1,2,1,48,64,0
2,3,0,36,70,1
3,4,1,15,72,1
4,5,0,14,36,0
...,...,...,...,...,...
95,96,0,27,30,0
96,97,1,21,62,1
97,98,1,43,47,0
98,99,1,48,79,1


In [9]:
# convert pandas dataframe object to Biogeme Database
database = db.Database('application', df)

In [10]:
# display database info
display(database.getSampleSize())
display(database.variables)

100

{'Individual': Individual,
 'Children_u16': Children_u16,
 'Car_travel_time': Car_travel_time,
 'PT_travel_time': PT_travel_time,
 'Chosen_alternative': Chosen_alternative}

In [11]:
# The following statement allows you to use the names of the
# variable as Python variable.
# specifiying the initial values 
globals().update(database.variables)

In [12]:
# Parameters to be estimated
# Arguments:
#   - 1  Name for report; Typically, the same as the variable.
#   - 2  Starting value.
#   - 3  Lower bound.
#   - 4  Upper bound.
#   - 5  0: estimate the parameter, 1: keep it fixed.

ASC_CAR = Beta('ASC_CAR', 0, None, None, 1) 
# 0 is the start value, 1 indicates whrther biversion is fixed, none is upper and lower limits
ASC_PT= Beta('ASC_PT', 0, None, None, 0)
B_TT = Beta('B_TT', 0, None, None, 0)
B_kids = Beta('B_kids', 0, None, None, 0)


In [13]:
# Definition of Utility functions
V0 = ASC_PT  +  B_TT*PT_travel_time
V1 = ASC_CAR +  B_TT*Car_travel_time + B_kids*Children_u16

In [14]:
# Associate utility functions with the numbering of alternatives
V = {0: V0, 1: V1}
# v1 v2 ... are the chosen traveling mode

In [15]:
# Associate the availability conditions with the alternatives
Av_PT=1
Av_CAR=1

av = {0:Av_PT,1:Av_CAR}

In [16]:
# The choice model is a logit, with availability conditions
logprob = models.loglogit(V, av, Chosen_alternative)

In [17]:
# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_with_biogeme'

File biogeme.toml has been created


In [20]:
# Estimate the parameters
results = biogeme.estimate()
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_PT,2.388567,0.771937,3.094252,1.973101e-03
B_TT,-0.120369,0.023719,-5.074728,3.880503e-07
B_kids,0.158518,0.671311,0.236132,8.133300e-01


In [21]:
df['Av_PT']=1
df['Av_Car']=1
df.to_csv('data.csv',index=False)